## ResNet
- 컨볼루션층의 출력에 전의 전 계층에 쓰였던 입력을 더함으로써 특징이 유실되지 않도록 함
- 여러 단계의 신경망을 거치며 최초 입력 이미지에 대한 정보가 소실되는 문제를 해결
- ResNet은 Residual 블록을 반복적으로 쌓은 것 뿐임
- CIFAR-10 데이터셋 사용 (10가지 분류 존재)

### Resnet 순서 설명
- 1. 입력이 들어오면 Conv -> BN -> activation 통과
- 2. BasicBlock 층을 갖고 있는 layer1, layer2, layer3을 통과
- 3. 각 layer는 2개씩의 Residual 블록을 가지고 있음
- 4. 평균 풀링을 하고 마지막 계층을 거쳐 분류 결과를 출력

![resnet](readme_image/resnet.png)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchvision import transforms, datasets

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

epochs = 10
batch_size = 64

train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/data', train=True, download=True,
                          transform = transforms.Compose([
                              transforms.RandomCrop(32, padding=4),
                              transforms.RandomHorizontalFlip(),
                              transforms.ToTensor(),
                              transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                          ])),
    batch_size = batch_size, shuffle = True
)

test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('/data', train=False, download = True,
                         transform = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
                         ])),
    batch_size = batch_size, shuffle = True
)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# 기본 공통 블록
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        
        # shortcut 모듈 정의
        self.shortcut = nn.Sequential() # nn.Module을 하나의 모듈로 묶는 역할.
        
        # resnet 2번쨰 블록부터 in_planes를 받아서, planes와 더해줌.
        if stride != 1 or in_planes !=planes:
            self.shortcut = nn.Sequential(
            nn.Conv2d(in_planes, planes,
                      kernel_size=1, stride = stride, bias = False),
            nn.BatchNorm2d(planes)
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # 기존 out에 shortcut 거친것 더하기
        out = F.relu(out)
        return out
    
class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes=16 # make_layers 함수에서 각 층을 만들 때 전층의 채널 출력값을 저장 layer1이 16이라 16으로 초기화
        
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(16, 2, stride=1) # 16 채널에서 16채널을 내보내는 BasicBlock 2개
        self.layer2 = self._make_layer(32, 2, stride=2) # 16 채널을 받아 32채널을 출력하는 BasicBlock 1개 / 32채널에서 32채널을 내보내는 BasicBlock 1개
        self.layer3 = self._make_layer(64, 2, stride=2) # 32 채널을 받아 64채널을 출력하는 BasicBlock 1개 / 64채널에서 64채널을 출력하는 BasicBlock 1개
        self.linear = nn.Linear(64, num_classes)
        
    def _make_layer(self, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1) # [1,1], [2,1], [2,1]
        layers =[]
        for stride in strides:
            layers.append(BasicBlock(self.in_planes, planes, stride))
            self.in_planes = planes
        return nn.Sequential(*layers)
    
    # 1. 입력이 들어오면 Conv -> BN -> activation 통과
    # 2. BasicBlock 층을 갖고 있는 layer1, layer2, layer3을 통과
    # 3. 각 layer는 2개씩의 Residual 블록을 가지고 있음
    # 4. 평균 풀링을 하고 마지막 계층을 거쳐 분류 결과를 출력
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x))) # conv - bn - relu
        out = self.layer1(out) # layer1 (residual 2개)
        out = self.layer2(out) # layer2 (residual 2개)
        out = self.layer3(out) # layer3 (residual 2개)
        out = F.avg_pool2d(out ,8) # 
        out = out.view(out.size(0), -1)
        out = self.linear(out) # 최종 결과 출력
        return out

In [13]:
def train(model, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE) # data, label DEVICE 로 DATA 보내기

        # 반복 때마다 기울기를 새로 계산하므로, optimizer.zero_grad() 함수 호출
        optimizer.zero_grad() # 최적화 함수에 대한 미분 진행 여부 설정
        output = model(data) # model 통과한 결과
        loss = F.cross_entropy(output, target) # target 과의 비교를 통한 cross entropy
        loss.backward() # 역방향 전파
        optimizer.step() # 미분 반복 진행
        
        if batch_idx % 200==0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx*len(data),
                                                                           len(train_loader.dataset),
                                                                           100.*batch_idx / len(train_loader), loss.item())
                 )

# 평가때는 기울기를 계산하지 않음
def evaluate(model, test_loader):
    model.eval() # 평가 모드로 모델 변경
    test_loss = 0 # 테스트 오차
    correct=0 # 예측이 맞은 수
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE) # test data, target DEVICE 로 전송
            output = model(data) 
            
            # 배치 오차 합산하기
            # 교차엔트로피시 미니배치의 합을 받아와야 함. 
            test_loss += F.cross_entropy(output, target, reduction='sum').item() # item() 함수 - 1개의 값을 가진 텐서의 값을 가져옴
        
            # 가장 높은 값을 가진 인덱스가 바로 예측값
            pred = output.max(1, keepdim=True)[1] # 가장 큰 확률 값을 가진 값 예측 -> 가장 큰 값과, 해당 값의 index 반환 -> index 사용
            correct += pred.eq(target.view_as(pred)).sum().item() # view_as => target을 pred의 shape처럼 바꿔줌 / sum().item() => 맞춘갯수의 합

    # 정확도 구하기
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    return test_loss, test_accuracy 


In [15]:
# to 함수는 모델의 파라미터들을 지정한 장치의 메모리로 보냄
model = ResNet().to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay =0.0005) # 학습률 감소 기법 사용
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)  # pytorch 내부의 lr_scheduler.steplr 도구
# step_size = 3 => 3번 호출될 때 학습률에 0.1 (gamma값을 곱함) -> 즉 3에폭후에 0.01 로 낮아짐

# 실제 학습 진행
for epoch in range(1, epochs+1):
    scheduler.step()
    train(model, train_loader, optimizer, epoch)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1 [0/50000 (0%)]	Loss: 2.480494
Train Epoch: 1 [12800/50000 (26%)]	Loss: 1.629405
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.521046
Train Epoch: 1 [38400/50000 (77%)]	Loss: 1.390602
[1] Test Loss: 1.4013, Accuracy: 48.96%
Train Epoch: 2 [0/50000 (0%)]	Loss: 1.217099
Train Epoch: 2 [12800/50000 (26%)]	Loss: 1.264889
Train Epoch: 2 [25600/50000 (51%)]	Loss: 1.298073
Train Epoch: 2 [38400/50000 (77%)]	Loss: 1.085313
[2] Test Loss: 1.4253, Accuracy: 54.48%
Train Epoch: 3 [0/50000 (0%)]	Loss: 1.052293
Train Epoch: 3 [12800/50000 (26%)]	Loss: 0.791506
Train Epoch: 3 [25600/50000 (51%)]	Loss: 0.965963
Train Epoch: 3 [38400/50000 (77%)]	Loss: 0.650558
[3] Test Loss: 0.8006, Accuracy: 71.50%
Train Epoch: 4 [0/50000 (0%)]	Loss: 0.859429
Train Epoch: 4 [12800/50000 (26%)]	Loss: 0.795063
Train Epoch: 4 [25600/50000 (51%)]	Loss: 0.753800
Train Epoch: 4 [38400/50000 (77%)]	Loss: 0.596752
[4] Test Loss: 0.7339, Accuracy: 74.28%
Train Epoch: 5 [0/50000 (0%)]	Loss: 0.656957
Train Epoch: 5 

In [16]:
print(model)

ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=